# Visualisation and Plotting of Experiments
> Run experiments first and save results into "./data/" directory

In [ ]:
import sys, os, pickle
import matplotlib.pyplot as P
import pandas as pd
import numpy as np
import seaborn as sns 
sys.path.append('../src')
from agent import AGENT, rolling_sum

In [ ]:
nHIDDEN = sorted(list(map(lambda x: int(x.split('-')[-1]), filter(lambda x: 'nHidden' in x, os.listdir('./data')))))
print(nHIDDEN)

In [ ]:
nExp = 10
DFs  = []
for nHidden in nHIDDEN:
    filename = f'./data/nHidden-{nHidden}'
    EXP = []
    for i in range(nExp):
        with open(f'{filename}/agent-{i}', 'rb') as f:
            EXP.append(pickle.load(f))
    data = np.array([rolling_sum(np.array(exp.df.reward), n = 20, normalise=True) for exp in EXP])
    dataP = [np.stack([row, np.arange(len(row)), np.repeat(i,len(row)), np.repeat(nHidden,len(row))]).T for i, row in enumerate(data)] # manipulate 2d array for DF-ready
    dataD = np.array(dataP).reshape(-1,4) # data for DF = np.array(dataP).reshape(-1,4) # data for DF
    df = pd.DataFrame(dataD, columns=['reward', 'episode', 'run', 'nHidden'])
    DFs.append(df)

In [ ]:
df         = pd.concat(DFs)
df.run     = df.run.astype('int64')
df.episode = df.episode.astype('int64')
df.nHidden = df.nHidden.astype('int64')
df

In [ ]:
sns.set_context("talk", font_scale=1.1)
palette = sns.cubehelix_palette(n_colors=len(nHIDDEN), start=2.1, rot=1, dark=0.25, light=0.7)

P.figure(figsize=(8,6))
sns.lineplot(data=df, x='episode', y='reward', hue='nHidden',ci=None, palette = palette);
P.savefig('./figs/reward-plot-nHidden-1.png',dpi=300, bbox_inches = 'tight', pad_inches = 0)